In [ ]:
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import OpenAI
from langchain_groq import ChatGroq 
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
pptx_ext = UnstructuredPowerPointLoader("AI First Step Theory.pptx")
pptx_ext = pptx_ext.load()
pptx_ext

[Document(metadata={'source': 'AI First Step Theory.pptx'}, page_content="Demystifying Generative AI\n\n1\n\n\n\nAI First Step Theory\n\n Machine Learning\n\nDeep Learning\n\nGenerative AI\n\nနက်ရှိုင်းသောအလွှာများကို အသုံးပြုပြီး ဒေတာမှစွမ်းဆောင်ရည်မြင့်စွာ သင်ယူနိုင်သည့် စက်သင်ယူနည်းပညာဖြစ်သည်။\n\nစက်တွေကို အတွေ့အကြုံမှ သင်ယူပြီး စနစ်တကျ လုပ်ဆောင်နိုင်အောင် လုပ်ခြင်းပါ။\n\nအသစ်သော စာသား၊ ပုံ၊ ဗီဒီယို စသဖြင့် အရာအသစ်များကို စက်က ဖန်တီးပေးနိုင်တဲ့ AI ဖြစ်ပါတယ်။\n\n2\n\n\n\nAI First Step Theory\n\n3\n\n\n\nUnderstanding Generative AI\n\nDefinition and Scope\n\nHow It Works\n\nKey Technologies\n\nGenerative AI refers to algorithms that can produce new content based on input data.\n\nTechnologies like GPT, DALL-E, and Codex represent the forefront of generative AI capabilities.\n\nThese systems utilize deep learning techniques, particularly neural networks, to analyze large datasets.\n\n4\n\n\n\nHow Generative AI works in background\n\n5\n\n\n\nApplications in Text Generation\n\nLanguage 

In [11]:
chunk = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=False)
split_doc = chunk.split_documents(pptx_ext)
split_doc

[Document(metadata={'source': 'AI First Step Theory.pptx'}, page_content='Demystifying Generative AI\n\n1\n\n\n\nAI First Step Theory\n\n Machine Learning\n\nDeep Learning\n\nGenerative AI\n\nနက်ရှိုင်းသောအလွှာများကို အသုံးပြုပြီး ဒေတာမှစွမ်းဆောင်ရည်မြင့်စွာ သင်ယူနိုင်သည့် စက်သင်ယူနည်းပညာဖြစ်သည်။\n\nစက်တွေကို အတွေ့အကြုံမှ သင်ယူပြီး စနစ်တကျ လုပ်ဆောင်နိုင်အောင် လုပ်ခြင်းပါ။\n\nအသစ်သော စာသား၊ ပုံ၊ ဗီဒီယို စသဖြင့် အရာအသစ်များကို စက်က ဖန်တီးပေးနိုင်တဲ့ AI ဖြစ်ပါတယ်။\n\n2\n\n\n\nAI First Step Theory\n\n3\n\n\n\nUnderstanding Generative AI\n\nDefinition and Scope\n\nHow It Works\n\nKey Technologies\n\nGenerative AI refers to algorithms that can produce new content based on input data.\n\nTechnologies like GPT, DALL-E, and Codex represent the forefront of generative AI capabilities.\n\nThese systems utilize deep learning techniques, particularly neural networks, to analyze large datasets.\n\n4\n\n\n\nHow Generative AI works in background\n\n5\n\n\n\nApplications in Text Generation\n\nLanguage 

In [17]:
client = QdrantClient(":memory:")
client.create_collection(
collection_name="zoomcamp-project1",
vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_store = QdrantVectorStore(
client=client,
collection_name="zoomcamp-project1",
embedding=embedding_model
)
model = ChatGroq(model_name="llama-3.3-70b-versatile")
vector_store.add_documents(split_doc)

retriever = vector_store.as_retriever(search_type="similarity_score_threshold",search_kwargs={"score_threshold": 0.5, "k": 3})

system_prompt = (
"You are a smart assistant. "
"Provide concise and professional responses from the given context.\n\n{context}"
)
prompt = ChatPromptTemplate.from_messages([
("system", system_prompt),
("human", "{input}")
])

In [18]:
stuff_chain = create_stuff_documents_chain(model, prompt)
chain = create_retrieval_chain(retriever, stuff_chain)

In [19]:
chain.invoke({"input":"what is this presentation in topics"})

{'input': 'what is this presentation in topics',
 'context': [Document(metadata={'source': 'AI First Step Theory.pptx', '_id': '17ea8360f4c64c01bef75aea2d0fc3e4', '_collection_name': 'zoomcamp-project1'}, page_content="18\n\n\n\nPrompts are instructions or inputs given to AI models that guide their responses.\n\nProviding context helps the AI understand background and intention, leading to relevant outputs.\n\nPrompts can be open-ended, specific, or directive, influencing AI's creativity and relevance.\n\nUnderstanding Prompts\n\nImportance of Context\n\nDefinition of Prompts\n\nTypes of Prompts\n\n19\n\n\n\nPrompt engineering often requires iterations. Testing different versions of prompts can help identify which ones yield the best results.\n\nProviding examples within prompts can significantly improve the quality of responses, guiding the AI on expected formats and content.\n\nEffective prompts should be clear and concise, avoiding ambiguity to ensure the AI understands what is bein